# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

### Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.  
Известно:  
сalls — количество звонков,    
minutes — суммарная длительность звонков в минутах,  
messages — количество sms-сообщений,  
mb_used — израсходованный интернет-трафик в Мб,  
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  

## Откройте и изучите файл

In [123]:
import pandas as pd
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier

In [124]:
df = pd.read_csv('/datasets/users_behavior.csv')
print("Размер исходной выборки:",df.shape)
print("Посмотрим первые 5 строк датасета:\n",df.head())
print("Информация о датасете:\n",df.info())
print("Число дубликатов:",df.duplicated().sum())
print("Сколько человек имеют тот или иной тариф:\n",df['is_ultra'].value_counts())

Размер исходной выборки: (3214, 5)
Посмотрим первые 5 строк датасета:
    calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
Информация о датасете:
 None
Число дубликатов: 0
Сколько человек имеют тот или иной тариф:
 0    2229
1     985
Name: is_ultra, dtype: int64


Количество смс и звонков - целые числа.
В данных отсутвуют пропуски и дубликаты, так как нам был предоставлен преобработанный датасет.

In [125]:
try:
    df[['calls','messages']] = df[['calls','messages']].astype('int64')
except:
    display('Не удалось  изменить тип данных столбцов calls,messages')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


## Разбейте данные на выборки

In [126]:
features =  df.drop(['is_ultra'], axis=1)
target = df['is_ultra'] 

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(1928, 4)
(643, 4)
(643, 4)


## Исследуйте модели

### Решающее дерево

In [127]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 15):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth,min_samples_split=80) 
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
print('Решающее дерево')
print("Accuracy лучшей модели на валидационной выборке:", round(best_result,3))
print('Максимальная глубина:', best_depth)

Решающее дерево
Accuracy лучшей модели на валидационной выборке: 0.807
Максимальная глубина: 8


Решающее дерево показало достаточно хорошую точность прогноза (0.807) на валидационной выборке при максимальной глубине дерева, равной 8. 

### Cлучайный лес

In [128]:
best_model = None
best_est = 0
best_depth = 0
best_result=0
for est in range(50, 201, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est)
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid) 
        result = accuracy_score(target_valid, predictions_valid)
        #result = model.score(features_valid, target_valid)
        if result > best_result:
            best_model = model
            #best_result = model.score(features_valid, target_valid)
            best_result = result
            best_depth=depth
            best_est = est
print('Cлучайный лес')
print("Accuracy лучшей модели на валидационной выборке:", round(best_result,3))
print('Максимальная глубина:', best_depth)
print("Количество деревьев:", best_est)



Cлучайный лес
Accuracy лучшей модели на валидационной выборке: 0.796
Максимальная глубина: 1
Количество деревьев: 80


Cлучайный лес показал достаточно хорошую точность прогноза (0.796) на валидационной выборке при максимальной глубине дерева, равной 1.Количество деревьев - 80. 

### Логистическая регрессия

In [129]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=3000,C=10) 
model.fit(features_train, target_train) 
model.predict(features_valid) 
predictions_valid = model.predict(features_valid) 
result=model.score(features_valid, target_valid) 
print('Логистическая регрессия')
print("Accuracy модели на валидационной выборке:", round(result,3))

Логистическая регрессия
Accuracy модели на валидационной выборке: 0.739


 Логистическая регрессия показала точность прогноза (0.739) на валидационной выборке.

Промежуточный вывод:   
По результатам проверки 3 моделей, пришла к выводу, что в данном случае решающее дерево имеет наибольшую точность и скорость работы. 

## (бонус) Проверьте модели на адекватность

In [130]:
model = DecisionTreeClassifier(random_state=12345, max_depth=9,min_samples_split=80) 
model.fit(features_train, target_train) 
predictions_test = model.predict(features_test) 
result = accuracy_score(target_test, predictions_test)
print("Решающее дерево предсказало тариф на тестовой выборке с точностью ",round(result,3))

Решающее дерево предсказало тариф на тестовой выборке с точностью  0.784


In [131]:
clf_dummy = DummyClassifier(random_state=12345) 
clf_dummy.fit(features_train, target_train) 
predictions_valid = clf_dummy.predict(features_valid) 
predictions_test = clf_dummy.predict(features_test) 
result_valid = accuracy_score(target_valid, predictions_valid)
result_test = accuracy_score(target_test, predictions_test)
print("Dummy модель предсказала тариф на тестовой выборке с точностью",round(result_test,3))

Dummy модель предсказала тариф на тестовой выборке с точностью 0.706


## Вывод:  
Для исследования нам была предложен преобработанный датасет с информацией о пользователях тарифов Смарт и Ультра. Мы изучили данные.   
Целью исследования было выявить модель классификации, которая максимально точно предскажет тариф клиента.  
Мы рассмотрели модели "Решающее дерево", "Случайный лес" и "Логистичная регрессия".   
Попытались подобрать оптимальные гиперпараметры с целью улучшить качество модели.   
В нашем случае решающее дерево показало высокую точность и скорость работы.     
Точность предсказаний на тестовой выборке равна 0.784.     
Сделали проверку модели на адекватность. Точность предсказания dummy моделью равна 0.706.   
То есть наша модель прошла проверку на адекватность. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
